# Fyzická kondice

Stáhni si data ze souboru [bodyPerformance.csv](bodyPerformance.csv) o fyzické kondici, která byla jihokorejskou organizací Korea Sports Promotion Foundation. Data obsahují různé vstupní hodnoty a výstupní hodnotu, kterou je zařazení jedince do jedné ze čtyř výkonnostních tříd.

- age = věk (20 až 64 let),
- gender = pohlaví (F, M),
- height_cm = výška v cm,
- weight_kg = hmotnost v kg,
- body_fat_% = tělesný tuk v procentech,
- distolic = diastolický krevní tlak (min),
- systolic = systolický krevní tlak (min),
- gripForce = síla stisku,
- sit and bend forward_cm = sed a předklon v cm,
- sit-ups counts = počty sedů-lehů,
- broad jump_cm = skok do dálky v cm,
- class  = třída fyzické výkonnosti (4 třídy, A je nejlepší a D nejhorší).

Uvažuj, že chceš přijímat lidi do organizace, která vyžaduje vysokou fyzickou výkonnost. Tvou snahou je zkrátit a zefektivnit přijímací proces. Zkus tedy zjistit, nakolik přesné je zařazení jedinců do výkonnostních tříd bez nutnosti měření jejich výkoknu při vykonání jednotlivých cviků. Využij tedy všechny vstupní proměnné s výjimkou `sit and bend forward_cm`, `sit-ups counts` a `broad jump_cm`.

- K rozřazení jedinců do skupin využij rozhodovací strom a jeden ze zbývajících dvou algoritmů probíraných na lekcích (tj. K Nearest Neighbours nebo Support Vector Machine). Rozhodovacímu stromu omez maximální počet pater na 5 a poté si zobraz graficky a vlož ho do Jupyter notebooku nebo jako obrázek ve formátu PNG jako součást řešení.
- Vytvoř matici záměn pro rozhodovací strom. Kolik jedinců s nejvyšší fyzickou výkonností (tj. ze skupiny A) bylo klasifikování správně? Kolik pak bylo zařazeno do skupin B, C a D? Uveď výsledky do komentáře v programu nebo do buňky v Jupyter notebooku.
- Urči metriku `accuracy` pro rozhodovací strom a pro jeden ze dvou vybraných algoritmů. Který algoritmus si vedl lépe? Odpověď napiš do komentáře.
- Nyní uvažuj, že se rozhodneš testovat jedince pomocí jednoho ze cviků. Vyber cvik, který dle tebe nejvíce vypovídá o fyzické výkonnosti jedince. Porovnej, o kolik se zvýšila hodnota metriky `accuracy` pro oba algoritmy.

## Nápovědy

Sloupec `gender` v našem datasetu vyžadují použítí `OneHotEncoder`, protože jde o kategoriální proměnné. Ostatní data je lepší nechat v původní hodnotě (protože budeme zatím pracovat s rozhodovacím stromem).

In [1]:
import pandas
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus
from pydotplus import graph_from_dot_data
from sklearn.preprocessing import StandardScaler
import numpy

import os
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'

data = pandas.read_csv("bodyPerformance.csv")
data.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


Níže je postup, jak nakombinovat informaci o pohlaví s číselnými sloupci. Postup vychází z postupu v lekcích. Podívej se na jednotlivé řádky a zkus vymyslet, jaký je jejich smysl.

In [ ]:
y = data["class"]

categorical_columns = ["gender"]
numeric_columns = ["age", "height_cm", "weight_kg", "body fat_%", "diastolic", "systolic", "gripForce"]
numeric_data = data[numeric_columns].to_numpy()

encoder = OneHotEncoder()
encoded_columns = encoder.fit_transform(data[categorical_columns])
encoded_columns = encoded_columns.toarray()

Níže je řádek s funkcí `export_graphviz()`. Ve funkci je důležité především nastavení `feature_names`, protože zde kombinujeme názvy sloupců v encoderu a názvy původních sloupců. K tomu použijeme operátor `+`, který spojí dva seznamy do jednoho. Pokud ti to není jasné, zkus si vypsat na obrazovku pouze výsledek "součtu" obou seznamů.

```py
export_graphviz(clf, out_file=dot_data, filled=True, feature_names=list(encoder.get_feature_names_out()) + numeric_columns, class_names=["A", "B", "C", "D"])
```

Níže je vykreslený rozhodovací strom. Tvůj strom určitě nemusí být stejný, ale při správném postupu bude podobný.

![napovedy_05/output.png](napovedy_05/output.png)

Hodnota metriky `accuracy` je 44.89 %.

Níže je kód, pomocí kterého můžeš využít `StandardScaler()` pro numerické sloupce a `OneHotEncoder()` pro sloupec s pohlavím.

In [ ]:
scaler = StandardScaler()
numeric_data = scaler.fit_transform(data[numeric_columns])

encoder = OneHotEncoder()
encoded_columns = encoder.fit_transform(data[categorical_columns])
encoded_columns = encoded_columns.toarray()

X = numpy.concatenate([encoded_columns, numeric_data], axis=1)

Pokud použiješ algoritmus K Nearest Neighbours, vyjde ti `accuracy` cca 41.04 %.